In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

from IPython.display import display, HTML, display_html
import tensorflow as tf

In [2]:
master = pd.read_csv('data/crimes_nat_tinker.csv')
master.drop(['Unnamed: 0','Unnamed: 0.1','SHOOTING'],axis=1,inplace=True)
master = pd.get_dummies(master, columns=['DAY_OF_WEEK','MONTH'])
master.neighborhood_avg = np.log(master.neighborhood_avg)
master.closest_property_value = np.log(master.closest_property_value)
master.head()

,Lat,Long,HOUR,TAVG,PRCP,SNOW,closest_property_value,neighborhood_avg,lamp_min_dist,lamp_density,category,police_min_dist,police_density,hospital_min_dist,hospital_density,school_min_dist,school_density,DAY_OF_WEEK_Friday,DAY_OF_WEEK_Monday,DAY_OF_WEEK_Saturday,DAY_OF_WEEK_Sunday,DAY_OF_WEEK_Thursday,DAY_OF_WEEK_Tuesday,DAY_OF_WEEK_Wednesday,MONTH_1,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12
0,42.259518,-71.121563,9,59.0,0.00,0.0,13.025852,12.996051,0.001234,563.0,theft,0.004067,1.0,0.041388,0.0,0.004665,4.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,42.262092,-71.116710,20,56.0,0.01,0.0,13.201729,13.023400,0.000539,531.0,theft,0.009414,1.0,0.036018,0.0,0.001244,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,42.352375,-71.135096,15,56.0,0.01,0.0,12.468052,13.287047,0.000167,1272.0,theft,0.015767,0.0,0.010692,0.0,0.002624,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,42.349476,-71.076402,4,56.0,0.01,0.0,13.106234,14.326388,0.000074,4461.0,theft,0.012329,0.0,0.012304,0.0,0.001918,6.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,42.381846,-71.066551,8,71.0,0.22,0.0,13.637460,13.313802,0.000108,2672.0,theft,0.011801,0.0,0.016170,0.0,0.002848,3.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [3]:
X, y = master.drop(['category'], axis=1), master['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)

In [4]:
def cat_to_num(x):
    if x == 'theft':
        return 0
    elif x == 'domestic':
        return 1
    elif x == 'violent':
        return 2
    elif x == 'medical':
        return 3
    elif x == 'vandalism':
        return 4
    elif x == 'petty theft':
        return 5
    elif x == 'mv accident':
        return 6
    elif x == 'drugs':
        return 7
    elif x == 'burglary ':
        return 8
    elif x == 'possession weapons':
        return 9
    elif x == 'civil disruption':
        return 10
    else:
        print(x)
    
# y_train_num = np.array(pd.get_dummies(y_train.apply(cat_to_num)))
# y_test_num = np.array(pd.get_dummies(y_test.apply(cat_to_num)))

y_train_num = np.array(y_train.apply(cat_to_num))
y_test_num = np.array(y_test.apply(cat_to_num))
np.unique(y_train_num)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [5]:
epochs = 20

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation='relu', input_shape=(35,)),
    tf.keras.layers.Dense(20, activation='relu'), # kernel_regularizer= tf.keras.regularizers.l2(l=0.1)),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(100, activation='relu'), # kernel_regularizer= tf.keras.regularizers.l2(l=0.1)),
    tf.keras.layers.Dense(11, activation='softmax')
])

# binary_crossentropy, categorical_crossentropy
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['categorical_accuracy'])

history_comp = model.fit(X_train, y_train_num, epochs=epochs, batch_size=128, validation_split=0.2, verbose=2)

Train on 157541 samples, validate on 39386 samples
Epoch 1/20
157541/157541 - 4s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 2/20
157541/157541 - 3s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 3/20
157541/157541 - 4s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 4/20
157541/157541 - 5s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 5/20
157541/157541 - 4s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 6/20
157541/157541 - 4s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accuracy: 0.0000e+00
Epoch 7/20
157541/157541 - 4s - loss: 3.2815 - categorical_accuracy: 0.0000e+00 - val_loss: 3.2902 - val_categorical_accu

In [6]:
train_acc = model.evaluate(X_train, y_train_num, verbose=0)
test_acc = model.evaluate(X_test, y_test_num, verbose=0)
print("Train Scores: " + str(train_acc))
print("Test Scores: " + str(test_acc))

Train Scores: [3.2832144083598984, 0.0]
Test Scores: [3.2631936512638076, 0.0]


In [7]:
model.predict(X_train)

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.2107004e-14, 1.3512486e-38, 1.9440348e-28, ..., 2.0302435e-17,
        1.0725649e-23, 2.1890893e-30],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)